Teste

In [1]:
print ("hello, world!")

hello, world!


In [2]:
%pip install --upgrade pip
%pip install python-constraint

from constraint import *


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Instanciar o Problema

In [3]:
tt_problem = Problem()
days = 'Segunda Terça Quarta Quinta Sexta'.split()
class_groups = 't01 t02 t03'.split()
blocks = 'B01 B02 B03 B04 B05 B06 B07 B08 B09 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B20'.split()
lecturers = 'jo mike rob sue'.split()
courses = 'UC11 UC12 UC13 UC14 UC15 UC21 UC22 UC23 UC24 UC25 UC31 UC32 UC33 UC34 UC35'.split()
rooms = 'A C G Lab01 Lab02'.split()


# Constrains

In [4]:

course_classes = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']
}
class_lecturers = {
    'jo' : ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35']
}
class_rooms = {
    'UC22' : ['Lab01'],
    'UC14' : ['Lab01'],
}

time_restrictions = {
    'mike' : ['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20'],
    'rob' : ['B01', 'B02', 'B03', 'B04'],
    'sue' : ['B09', 'B10', 'B11', 'B12', 'B17', 'B18', 'B19', 'B20']
    }

online_lessons = {'UC21': {2}, 'UC31': {2}}

days_blocks = {
    'Segunda': ['B01', 'B02', 'B03', 'B04'],
    'Terça': ['B05', 'B06', 'B07', 'B08'],
    'Quarta': ['B09', 'B10', 'B11', 'B12'],
    'Quinta': ['B13', 'B14', 'B15', 'B16'],
    'Sexta': ['B17', 'B18', 'B19', 'B20'],
}

online_by_lesson = {"UC21_2", "UC31_2"}




# Definir as Variáveis

In [5]:
# ——— 1) Dados auxiliares ———
# #oc: só a 2.ª aula destas UCs é online
online_by_lesson = {"UC21_2", "UC31_2"}

# Aulas a agendar (2 por UC)
lessons_to_schedule = [f"{uc}_{i}" for uc in courses for i in (1, 2)]

# Inversões (UC -> docente; UC -> turma)
course_teacher = {uc: t for t, ucs in class_lecturers.items() for uc in ucs}
course_group   = {uc: g for g, ucs in course_classes.items()   for uc in ucs}

# ——— 2) Domínio por AULA ———
def domain_for_lesson(lesson):
    uc = lesson.split("_")[0]
    teacher = course_teacher[uc]
    blocked = set(time_restrictions.get(teacher, []))
    blocks_ok = [b for b in blocks if b not in blocked]

    if lesson in online_by_lesson:
        room_list = ["Online"]             # online não ocupa sala física
    elif uc in class_rooms:
        room_list = class_rooms[uc]        # ex.: UC14/UC22 -> ["Lab01"]
    else:
        room_list = rooms                  # qualquer sala física

    return [(b, r) for b in blocks_ok for r in room_list]

# ——— 3) Criar o problema “limpo” e as variáveis uma vez ———
tt_problem = Problem()
for lesson in lessons_to_schedule:
    tt_problem.addVariable(lesson, domain_for_lesson(lesson))  # <— passa lesson, não uc

# ——— 4) Constraints ———
def all_diff_blocks(*vals):
    # Impede choques de bloco (conta online também)
    return len({b for (b, _) in vals}) == len(vals)

def no_same_room_block(*vals):
    # Uma aula por (sala,bloco); ignora "Online"
    phys = [(b, r) for (b, r) in vals if r != "Online"]
    return len(set(phys)) == len(phys)

block_to_day = {b: d for d, bls in days_blocks.items() for b in bls}
def different_day(a, b):
    # As 2 aulas da mesma UC em dias diferentes
    return block_to_day[a[0]] != block_to_day[b[0]]

# 4a) Por docente
for t in lecturers:
    vars_t = [f"{uc}_{i}" for uc in class_lecturers[t] for i in (1, 2)]
    tt_problem.addConstraint(all_diff_blocks, tuple(vars_t))

# 4b) Por turma
for g in class_groups:
    vars_g = [f"{uc}_{i}" for uc in course_classes[g] for i in (1, 2)]
    tt_problem.addConstraint(all_diff_blocks, tuple(vars_g))

# 4c) Por sala (global)
tt_problem.addConstraint(no_same_room_block, tuple(lessons_to_schedule))

# 4d) Duas aulas da mesma UC em dias distintos
for uc in courses:
    tt_problem.addConstraint(different_day, (f"{uc}_1", f"{uc}_2"))

# ——— 5) Resolver ———
solution = tt_problem.getSolution()
print(solution)




KeyboardInterrupt: 